In [112]:
import pandas as pd
import numpy as np
import ast

In [34]:
df = pd.read_csv("D:/DOWNLOADS/finalDataset.csv")
df.head()

,userID,movieID,rating,title,imdbID,keywords
0,8624,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
1,8930,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
2,12028,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
3,13916,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
4,15264,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."


In [103]:
n_data = pd.read_csv("D:/DOWNLOADS/movieNeighbors.csv")
n_data.head()

,Unnamed: 0,keys,values
0,0,Grumpy Old Men,"['Kind Hearts and Coronets', 'Song of the Sout..."
1,1,Twelve Monkeys,"['Breach', 'Outbreak', 'Meet the Robinsons', '..."
2,2,Braveheart,"['Local Hero', 'The Water Horse', 'Beverly Hil..."
3,3,Congo,"['Elektra', 'The Hand That Rocks the Cradle', ..."
4,4,Desperado,"['The Matrix Revolutions', 'Saw III', 'Sånger ..."


In [36]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def score(i, S, L, R):
    eta = 0.5
    overlap_S = overlap(i, S, R)
    overlap_L_minus_S = overlap(i, set(L) - set(S), R)
    return (1 - eta) * overlap_S + eta * overlap_L_minus_S

def overlap(i, X, R):
    X = set(X)
    covered = set(item for sublist in [Ni(j) for j in R] for item in sublist)
    numerator = 2 * len((Ni(i) - covered).intersection(X))
    denominator = len(Ni(i) - covered) + len(X - covered)
    return numerator / denominator if denominator > 0 else 0

In [37]:
def argmax(Candidates, S, L, R):
    d=[]
    for i in Candidates:
        d.append([i,score(i, S, L, R)])
    d = sorted(d, key = lambda x: int(x[1]))
    i_star = d[0][0]
    return i_star

In [38]:
user_profile = df[df['userID']==8624]
keywords = df['keywords'].unique()
movies = df['title'].unique()
print(user_profile.shape)
seen = user_profile['title'].tolist()
L=[]
D=dict()
c=0
for i in user_profile.values:
    c=c+1
    #print("COUNT:",c)
    li=[]
    mi = i[3]
    try:
        tag = i[-1].split(',')
        tag = set(tag)
        for k in keywords:
            try:
                tm = set(k.split(','))
                sim = jaccard_similarity(tag, tm)
                #print(tag, tm)
                if(sim>0):
                    li.append(k)
                    L.append(k)
            except:
                #print("No tag found")
                continue
        D[mi] = li
    except:
        continue
         
N=dict()
L=[]
for i in D:
    li=[]
    for k in D[i]:
        mov = df[df['keywords']==k]['title'].tolist()
        mov = list(set(mov))
        for m in mov:
            if(m!=i and m not in seen):
                li.append(m)
                L.append(m)
    N[i] = li
L = list(set(L))   

(95, 6)


In [39]:
def Nj(i):
    ke=[]
    try:
        tag = set(df[df['title']==i]['keywords'].tolist()[0].split(','))
    except:
        return set([])
    for k in keywords:
        try:
            tm = set(k.split(','))
            sim = jaccard_similarity(tag, tm)
            if(sim>0):
                ke.append(k)
        except:
            continue
    ke = list(set(ke))
    li = []
    for k in ke:
        m = df[df['keywords']==k]['title'].tolist()
        li = li + m 
    return set(li)
        
    

In [115]:
def Ni(i):
    li = n_data[n_data['keys']==i]['values'].tolist()[0]
    li = ast.literal_eval(li)
    return set(li)

In [116]:
Ni('Song of the South')

{"'Breaker' Morant",
 'Above the Rim',
 'Akeelah and the Bee',
 'An American Tail',
 'Around the World in 80 Days',
 'Candyman',
 'Dances with Wolves',
 'Devil in a Blue Dress',
 'DysFunktional Family',
 'Fletch Lives',
 'Ghost Dog: The Way of the Samurai',
 'Glory',
 'Grumpy Old Men',
 'Holes',
 'In the Heat of the Night',
 'Lean on Me',
 'Malcolm X',
 'Men of Honor',
 'Mississippi Burning',
 'Murder at 1600',
 'Nothing But Trouble',
 'Nothing to Lose',
 'Roger & Me',
 'Save the Last Dance',
 'Set It Off',
 'Sgt. Bilko',
 'Song of the South',
 'Tasogare Seibei',
 'The Boys of Baraka',
 'The Color Purple',
 'The Old Man and the Sea',
 'They Died with Their Boots On',
 'Van Helsing',
 'Vanity Fair',
 'Watership Down',
 'Who Framed Roger Rabbit',
 'Zardoz'}

In [41]:
user_profile[:10]

,userID,movieID,rating,title,imdbID,keywords
0,8624,3,4.5,Grumpy Old Men,107050,"old man,feud,old age,neighbor neighbor relatio..."
10118,8624,29,4.5,La cité des enfants perdus,112682,"steampunk fantasy,steampunk sci fi,orphanage,o..."
12021,8624,318,5.0,The Shawshank Redemption,111161,"prison,based on the works of stephen king,esca..."
13415,8624,357,5.0,Four Weddings and a Funeral,109831,"wedding,funeral,gay character,cafe,hotel"
15756,8624,671,5.0,Mystery Science Theater 3000: The Movie,117128,"robot sci fi,outer space,mad scientist,b movie..."
16066,8624,745,5.0,Wallace and Gromit in A Close Shave,112691,0
20259,8624,1079,5.0,A Fish Called Wanda,95159,"stuttering man,heist,ugly american,fish tank,h..."
20444,8624,1080,5.0,Life of Brian,79470,"monty python,actor playing multiple roles,sati..."
21863,8624,1136,5.0,Monty Python and the Holy Grail,71853,"killer rabbit,monty python,holy grail,breaking..."
22381,8624,1148,5.0,Wallace & Gromit in The Wrong Trousers,108598,"dog,stop motion animation,angry mob,animal dri..."


In [42]:
L[:10]

['The Age of Innocence',
 'Batoru rowaiaru',
 'Frenzy',
 'Duel',
 'The Grapes of Wrath',
 'The United States of Leland',
 'Capote',
 'Key Largo',
 'Primal Fear',
 'Dip huet seung hung']

In [122]:
def update(s, R, pi = "Open"):
    if(pi=="Open"):
        S = Ni(s)
        return S
    else:
        return {}

In [123]:
def nbyip(s, L, pi = "Open", n=2):
    S = N[s]
    Tabu = [s]
    Tabu = set(Tabu)
    while len(S)>n:
        R = recommend(S, L, n)
        print(R)
        print("Liked Item: ")
        s = input()
        print("Stop/Continue: ")
        a = input()
        if(a=="Stop"):
            return s
        R = R - set(s)
        S = update(s, R, pi)
        Tabu = Tabu.union(R)
        S = S - Tabu

In [120]:
def recommend(S, L, n=2):
    Candidates = set(S)
    R = set([])
    while len(R) < n and len(Candidates) > 0:
        #print(R)
        i_star = argmax(Candidates, S, L, R)
        #print(i_star)
        R.add(i_star)
        Candidates.remove(i_star)
        #print(Candidates)
    return R

In [121]:
nbyip("Grumpy Old Men", L, n=5)

{'Kind Hearts and Coronets', 'Song of the South', 'Sgt. Bilko', 'Richard III', 'Nothing But Trouble'}
Liked Item: 
Song of the South
Stop/Continue: 
Continue
{'DysFunktional Family', 'Akeelah and the Bee', 'Mississippi Burning', 'Glory', 'Fletch Lives'}
Liked Item: 
Mississippi Burning
Stop/Continue: 
Stop


'Mississippi Burning'

In [57]:
N["Grumpy Old Men"]

['Richard III',
 'The Old Man and the Sea',
 'Key Largo',
 'Kind Hearts and Coronets',
 'Zardoz',
 'Hello, Dolly!',
 'Private Benjamin',
 "Widows' Peak",
 'Driving Miss Daisy',
 'Song of the South',
 'Nothing But Trouble',
 'Sgt. Bilko']

In [100]:
data=dict()
dak=[]
dav=[]
print(len(movies))
c=0
for m in movies:
    c=c+1
    if(c%100==0):
        print(c)
    dav.append(list(Nj(m)))
    dak.append(m)
data['keys'] = dak
data['values'] = dav


6479
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400


In [101]:
data

{'keys': ['Grumpy Old Men',
  'Twelve Monkeys',
  'Braveheart',
  'Congo',
  'Desperado',
  'Die Hard: With a Vengeance',
  'Judge Dredd',
  'Pulp Fiction',
  'The Crow',
  'Beverly Hills Cop III',
  'Terminator 2: Judgment Day',
  'DragonHeart',
  'Ransom',
  'Gone with the Wind',
  'Last Man Standing',
  'Die Hard',
  'The Abyss',
  'Army of Darkness',
  'Das Boot',
  'Butch Cassidy and the Sundance Kid',
  'Star Trek: The Wrath of Khan',
  'Liar Liar',
  'The Fifth Element',
  'Armageddon',
  'Back to the Future Part II',
  'Honey, I Shrunk the Kids',
  'The Negotiator',
  'Payback',
  'The Matrix',
  'Superman',
  "The General's Daughter",
  'South Park: Bigger Longer & Uncut',
  'The Sixth Sense',
  'Fight Club',
  'Death Becomes Her',
  'X-Men',
  'Highlander: Endgame',
  'Unbreakable',
  'The Lord of the Rings: The Fellowship of the Ring',
  "Hart's War",
  'Dog Soldiers',
  'The Lord of the Rings: The Two Towers',
  'Tears of the Sun',
  'X2',
  'Hudson Hawk',
  'The Last Boy S

In [102]:
data = pd.DataFrame(data)
data.to_csv("D:/DOWNLOADS/movieNeighbors.csv")

In [97]:
data['values']

[]